# 0. 準備

## 0.1. スライドとデータ
* データ （前回同様）CSV形式：[exp_data.csv](https://github.com/HumanomeLab/TMDU_lecture_2019/blob/master/data/exp_data.csv)

## 0.2 GPUの有効化

以下の手順でGPUを選択する。
* 英語：Edit > Notebook Settings > Hardware Accelerator 
* 日本語：編集 > ノートブックの設定 > ハードウェアアクセレレータ

今回は、ほとんど効果が無いですが・・・

## 0.3 データのアップロード

前回同様、データをアップロードする。


# 1. NNを用いたクラス分類2 (PyTorch)

PyTorchで深層学習を実装をしていく。



## 1.1 前準備

### 1.1.1 パッケージとデータの準備

In [0]:
# PyTorch で用いられるクラス群
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import transforms
import torch.utils.data as data
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

import os
import copy
import time
import numpy as np
import pandas as pd

exp_data.csvに対して、以下のことを実施する make_dataset 関数を作成する。
1. データの読み込み
2. クラスを以下のベクトル形式で指定する　（one-hot vector と呼ばれる）
  * クラスが0 (y==0) なら、\[1, 0\]
  * クラスが1 (y==1) なら、\[0, 1\]

In [0]:
# データ読み込み関数を作成する。
def make_dataset():
    labels = []
    # データの読み込みと特徴量の選択
    dataset = pd.read_csv("exp_data.csv", index_col="Name")
    df = dataset.drop("class", axis=1)
    # df の値を取り出し、型を変換する
    X = df.values.astype(np.float32)
    y = np.array(dataset["class"].values)
    # クラスの値を one-hot-vector に変換する
    for y_i in y:
        c = [0, 0]  # one-hot-vector
        if y_i == 0.0:
            c = [1, 0]
        else:
            c = [0, 1]
        labels.append(np.array(c))
    return X, labels

データの読み込み。訓練データ、テストデータに加えて、バリデーションデータも作成する。深層学習では、正しく学習が進んでいるかどうかの判断に、バリデーションデータを利用する。
訓練データ、テストデータ、バリデーションデータの分割は、いずれも train_test_split を利用する。

In [0]:
# 全データの読み込み
X, y = make_dataset()
# テストデータの分割
X_tmp, X_test, y_tmp, y_test = train_test_split(X, y, test_size = 0.20)
# 訓練データとValidationデータの分割
X_train, X_val, y_train, y_val = train_test_split(X_tmp, y_tmp, test_size = 0.25)

正しく分割できているか確認するため、サンプル数を表示してみる。

In [22]:
print("X:", X.shape)
print("X_train:", X_train.shape)
print("X_val:", X_val.shape)
print("X_test:", X_test.shape)

X: (994, 906)
X_train: (596, 906)
X_val: (199, 906)
X_test: (199, 906)


DatasetFolderクラスを定義する。各サンプル点の変数をsampleに、クラスをtargetsに保持する。

In [0]:
# データを入れておくための倉庫
class DatasetFolder(data.Dataset):
    def __init__(self, X, y):
        self.samples = X
        self.targets = y

    def __getitem__(self, index):
        # index番目の変数とクラスを返す
        sample = self.samples[index]
        target = self.targets[index]
        sample = torch.from_numpy(sample)  # numpyからPyTorchの型へ変換
        target = torch.from_numpy(target)  # numpyからPyTorchの型へ変換
        return sample, target

    def __len__(self):
        return len(self.samples)

# 訓練データ, バリデーションデータ, テストデータをそれぞれ, 
# 'train', 'val', 'test' に保持する辞書を作成
# 例えば、 feature_datasets['train'] で訓練データのリストが得られる
feature_datasets = {
    'train':DatasetFolder(X_train, y_train),
    'val':DatasetFolder(X_val, y_val),
    'test': DatasetFolder(X_test, y_test)
}

訓練データ、バリデーションデータ、テストデータのデータの読み込み方（バッチサイズ、シャッフルの有無など）を指定する。

In [0]:
# バッチサイズ分のデータを読み込むための準備。
# 訓練データ（train）はshuffle=Trueを指定することで、
# データの順番をシャッフルし、読み込む順番をランダムにする。
# 他はシャッフルの必要なし。
batch_size=32
workers=0
dataloaders = {
    # 訓練データ
    'train': torch.utils.data.DataLoader(
        feature_datasets['train'],
        batch_size=batch_size,
        shuffle=True,
        num_workers=workers),
    # バリデーションデータ
    'val': torch.utils.data.DataLoader(
        feature_datasets['val'],
        batch_size=batch_size,
        shuffle=False,
        num_workers=workers),
    # テストデータ
    'test': torch.utils.data.DataLoader(
        feature_datasets['test'],
        batch_size=batch_size,
        shuffle=False,
        num_workers=workers)
}

# 訓練, バリデーション, テストデータのサンプル数を数える
dataset_sizes = {x: len(feature_datasets[x]) for x in ['train', 'val', 'test']}

### 1.1.2 ネットワークの定義

入力層のノード数が906 (全遺伝子分)、中間層のノード数が64のネットワークを作成する。

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(906, 64)
        self.fc2 = nn.Linear(64, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# ここから先は、作成したネットワークを、指定のデバイスに送るための内容。
# 今回はGPUが利用できる（ように設定している）ので、torch.cuda.is_available() が
# true となり、cuda （GPU）にデータが送られて計算される。
# CPUしか無い場合には、CPU上で計算。
device_name = "cpu"
if torch.cuda.is_available():
    device_name = "cuda"
device = torch.device(device_name)
model = Net()
model = model.to(device)

### 1.1.3 作成したネットワークの確認

自身が作成したネットワークが意図しているものと同じかどうかを確かめるため、表示する。PyTorchでは、torchsummary を用いるとモデルの概要を出力できる。

In [26]:
from torchsummary import summary
summary(model, (1,906))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                [-1, 1, 64]          58,048
            Linear-2                 [-1, 1, 2]             130
Total params: 58,178
Trainable params: 58,178
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.22
Estimated Total Size (MB): 0.23
----------------------------------------------------------------


### 1.1.4 モデルを訓練する関数の定義
モデルを訓練する train_model を定義する。
この関数は、大きく２つのfor文でできている。

"for epoch in range()"の中は、1エポックの実行である（12〜60行目）。各ループの最後に、更新したモデルがバリデーションデータの上で精度が良くなっているか確認し（55行目）、よくなっていればモデルを保存する（56、57行目）。

"for inputs, labels in dataloaders..."の中は１つのバッチの実行である（26〜46行目）。今のモデルを利用したときの目的関数（スライドのL）の値の計算（38行目）、勾配降下法を実施（41, 42行目）を行う。

In [0]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()
    # 途中経過でモデル保存するための初期化
    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss = 10000.0
    # 時間計測用
    end = time.time()
    
    # ネットワークの表示
    print(model)
    print()

    for epoch in range(num_epochs):
        print('Epoch:{}/{}'.format(epoch, num_epochs - 1), end="")

        # 各エポックで訓練+バリデーションを実行
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train(True)  # training mode
            else:
                model.train(False)  # evaluate mode

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                labels = labels.float()
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                # 訓練のときだけ履歴を保持する
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, classnums = torch.max(labels, 1)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, classnums)
                    # 訓練時にはバックプロパゲーションを実施
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # 統計情報
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == classnums)
                
            # サンプル数で割って平均を求める
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('\t{} Loss: {:.4f} Acc: {:.4f} Time: {:.4f}'.format(phase, epoch_loss, epoch_acc, time.time()-end), end="")
            
            # 学習が進むに連れ、更新量を減らすための処理
            if phase == 'train':
                scheduler.step()
            # 精度が改善したらモデルを保存する
            if phase == 'val' and epoch_loss < best_loss:
                best_loss = epoch_loss
                best_model_wts = copy.deepcopy(model.state_dict())
            end = time.time()

        print()

    time_elapsed = time.time() - since
    print()
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val acc: {:.4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

### 1.1.5 テストデータにおける評価のための関数

学習したモデルについてテストデータにおける精度を調査するための関数を定義する。ほとんど訓練時と同じ関数だが、バックプロパゲーションが無いなど、簡素化されている。

In [0]:
# テストデータの評価
def print_test_accuracy(model, criterion, phase):
    running_loss = 0.0
    running_corrects = 0
    model.train(False)

    for inputs, labels in dataloaders[phase]:
        labels = labels.float()
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        # 訓練のときだけ履歴を保持する
        with torch.set_grad_enabled(phase == 'train'):
            outputs = model(inputs)
            _, classnums = torch.max(labels, 1)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, classnums)
        # 統計情報
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == classnums)

    # サンプル数で割って平均を求める
    epoch_loss = running_loss / dataset_sizes[phase]
    epoch_acc = running_corrects.double() / dataset_sizes[phase]
    print('On Test:\tLoss: {:.4f} Acc: {:.4f}'.format(epoch_loss, epoch_acc))

## 1.2 モデルの学習と評価

### 1.2.1 モデルの学習
目的関数を交差エントロピーで、確率的勾配降下法でパラメータを更新し、モデルを学習する。また、テストデータを用いて予測精度を計算する。

In [30]:
epochs = 40
batch_size = 32
lr = 0.001
momentum = 0.9
outdir = "."

# 目的関数（クロスエントロピー）、
# パラメータの最適化方法、学習率の更新方法を定義。
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.7)

# 実際の学習を実施
model = train_model(model, criterion, optimizer, exp_lr_scheduler, num_epochs=epochs)
# テストデータでの精度を求める
print_test_accuracy(model, criterion,  'test')

Net(
  (fc1): Linear(in_features=906, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=2, bias=True)
)

Epoch:0/39	train Loss: 0.6483 Acc: 0.8557 Time: 0.0410	val Loss: 0.6316 Acc: 0.7337 Time: 0.0077
Epoch:1/39	train Loss: 0.6146 Acc: 0.8121 Time: 0.0393	val Loss: 0.5952 Acc: 0.5879 Time: 0.0066
Epoch:2/39	train Loss: 0.5666 Acc: 0.8003 Time: 0.0310	val Loss: 0.5351 Acc: 0.9397 Time: 0.0064
Epoch:3/39	train Loss: 0.5235 Acc: 0.8440 Time: 0.0299	val Loss: 0.4805 Acc: 0.8442 Time: 0.0058
Epoch:4/39	train Loss: 0.4939 Acc: 0.8473 Time: 0.0350	val Loss: 0.4650 Acc: 0.9497 Time: 0.0065
Epoch:5/39	train Loss: 0.4455 Acc: 0.9010 Time: 0.0310	val Loss: 0.4300 Acc: 0.9548 Time: 0.0064
Epoch:6/39	train Loss: 0.4338 Acc: 0.8893 Time: 0.0320	val Loss: 0.4094 Acc: 0.7889 Time: 0.0062
Epoch:7/39	train Loss: 0.4335 Acc: 0.8859 Time: 0.0311	val Loss: 0.4266 Acc: 0.9397 Time: 0.0063
Epoch:8/39	train Loss: 0.4008 Acc: 0.8993 Time: 0.0307	val Loss: 0.3472 Acc: 0.9347 Time: 0.0064

## 1.3 モデルの変更 (Dropoutを追加する)

過学習傾向にある可能性があるので、過学習を避けるために、Dropoutを実施する。中間層でDropoutを実施する。
Dropout とは、ランダムにネットワークを間引いて学習することで、過学習を避ける際に利用される方法の一つ。

In [0]:
# ネットワークの定義
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(906, 64)
        self.fc2 = nn.Linear(64, 2)
        self.drop = nn.Dropout(p=0.5)  # 確率0.5でdropoutすることを宣言

    def forward(self, x):
        x = F.relu(self.drop(self.fc1(x)))  # Dropoutの実施
        x = self.fc2(x)
        return x

# ネットワークをGPUに送る
model = Net()
model = model.to(device)  

In [32]:
summary(model, (1,906))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                [-1, 1, 64]          58,048
           Dropout-2                [-1, 1, 64]               0
            Linear-3                 [-1, 1, 2]             130
Total params: 58,178
Trainable params: 58,178
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.22
Estimated Total Size (MB): 0.23
----------------------------------------------------------------


再度、学習の実施。

In [33]:
epochs = 20
batch_size = 32
lr = 0.001
momentum = 0.9
outdir = "."

# 損失関数（クロスエントロピー）、
# パラメータの最適化方法、学習率の更新方法を定義。
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.7)

# 実際の学習を実施
model = train_model(model, criterion, optimizer, exp_lr_scheduler, num_epochs=epochs)
# テストデータでの精度を求める
print_test_accuracy(model, criterion, 'test')

Net(
  (fc1): Linear(in_features=906, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=2, bias=True)
  (drop): Dropout(p=0.5, inplace=False)
)

Epoch:0/19	train Loss: 0.7624 Acc: 0.5050 Time: 0.0409	val Loss: 0.7111 Acc: 0.5025 Time: 0.0081
Epoch:1/19	train Loss: 0.6759 Acc: 0.5721 Time: 0.0339	val Loss: 0.6775 Acc: 0.4975 Time: 0.0061
Epoch:2/19	train Loss: 0.6474 Acc: 0.6040 Time: 0.0339	val Loss: 0.6229 Acc: 0.7286 Time: 0.0066
Epoch:3/19	train Loss: 0.6147 Acc: 0.6829 Time: 0.0333	val Loss: 0.6016 Acc: 0.6784 Time: 0.0065
Epoch:4/19	train Loss: 0.6010 Acc: 0.6762 Time: 0.0362	val Loss: 0.5330 Acc: 0.9447 Time: 0.0061
Epoch:5/19	train Loss: 0.5056 Acc: 0.7768 Time: 0.0335	val Loss: 0.4427 Acc: 0.9296 Time: 0.0063
Epoch:6/19	train Loss: 0.4434 Acc: 0.8456 Time: 0.0386	val Loss: 0.4083 Acc: 0.8693 Time: 0.0078
Epoch:7/19	train Loss: 0.4009 Acc: 0.8456 Time: 0.0326	val Loss: 0.3047 Acc: 0.9548 Time: 0.0062
Epoch:8/19	train Loss: 0.4320 Acc: 0.7953 Time: 0.0336	v

層の数を増やしてみる。

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(906, 32)
        self.fc2 = nn.Linear(32, 16)
        self.fc3 = nn.Linear(16, 8)
        self.fc4 = nn.Linear(8, 2)
        self.drop = nn.Dropout(p=0.5)

    def forward(self, x):
        x = F.relu(self.drop(self.fc1(x)))
        x = F.relu(self.drop(self.fc2(x)))
        x = F.relu(self.drop(self.fc3(x)))
        x = self.fc4(x)
        return x

model = Net()
model = model.to(device)

In [35]:
summary(model, (1,906))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                [-1, 1, 32]          29,024
           Dropout-2                [-1, 1, 32]               0
            Linear-3                [-1, 1, 16]             528
           Dropout-4                [-1, 1, 16]               0
            Linear-5                 [-1, 1, 8]             136
           Dropout-6                 [-1, 1, 8]               0
            Linear-7                 [-1, 1, 2]              18
Total params: 29,706
Trainable params: 29,706
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.11
Estimated Total Size (MB): 0.12
----------------------------------------------------------------


In [37]:
epochs = 50
batch_size = 32
lr = 0.001
momentum = 0.9
outdir = "."

# 損失関数（クロスエントロピー）、
# パラメータの最適化方法、学習率の更新方法を定義。
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.7)

# 実際の学習を実施
model = train_model(model, criterion, optimizer, exp_lr_scheduler, num_epochs=epochs)

Net(
  (fc1): Linear(in_features=906, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=16, bias=True)
  (fc3): Linear(in_features=16, out_features=8, bias=True)
  (fc4): Linear(in_features=8, out_features=2, bias=True)
  (drop): Dropout(p=0.5, inplace=False)
)

Epoch:0/49	train Loss: 0.6908 Acc: 0.5336 Time: 0.0631	val Loss: 0.6933 Acc: 0.4975 Time: 0.0082
Epoch:1/49	train Loss: 0.6959 Acc: 0.5352 Time: 0.0515	val Loss: 0.6936 Acc: 0.4975 Time: 0.0080
Epoch:2/49	train Loss: 0.6955 Acc: 0.5134 Time: 0.0490	val Loss: 0.6938 Acc: 0.4975 Time: 0.0077
Epoch:3/49	train Loss: 0.6925 Acc: 0.5235 Time: 0.0520	val Loss: 0.6938 Acc: 0.4975 Time: 0.0078
Epoch:4/49	train Loss: 0.6944 Acc: 0.5252 Time: 0.0571	val Loss: 0.6938 Acc: 0.4975 Time: 0.0079
Epoch:5/49	train Loss: 0.6959 Acc: 0.5050 Time: 0.0467	val Loss: 0.6939 Acc: 0.4975 Time: 0.0075
Epoch:6/49	train Loss: 0.6950 Acc: 0.5050 Time: 0.0471	val Loss: 0.6940 Acc: 0.4975 Time: 0.0073
Epoch:7/49	train Loss: 0.6908 Acc: 

In [38]:
# テストデータでの精度を求める
print_test_accuracy(model, criterion, 'test')

On Test:	Loss: 0.6153 Acc: 0.9246


以上で、発現量を利用した深層ではないニューラルネットワークを実施した。層の数や層内のノードの数など、様々な変更可能な点が存在しているので、色々と試してみてほしい。

# 2. 学習経過の表示

In [76]:
%load_ext tensorboard
%tensorboard --logdir ./runs

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 1546), started 1:18:53 ago. (Use '!kill 1546' to kill it.)

In [80]:
from tensorboardcolab import TensorBoardColab
tbc = TensorBoardColab()    

Wait for 8 seconds...
TensorBoard link:
https://224b6986.ngrok.io


表示されるURLをクリックして、タブを開いておいてください。
後ほど、学習のグラフが描かれます。

In [0]:
# ネットワークの定義。1.1.2と同様
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(906, 64)
        self.fc2 = nn.Linear(64, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# 学習の定義。変更点は、
# 1. Tensorboard用の表記, 2. わざとepochごとに3秒のインターバルを入れた
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()
    # 途中経過でモデル保存するための初期化
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    # 時間計測用
    end = time.time()
    
    # ネットワークの表示
    print(model)
    print()

    for epoch in range(num_epochs):
        print('Epoch:{}/{}'.format(epoch, num_epochs - 1), end="")

        # 各エポックで訓練+バリデーションを実行
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train(True)  # training mode
            else:
                model.train(False)  # evaluate mode

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                labels = labels.float()
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                # 訓練のときだけ履歴を保持する
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, classnums = torch.max(labels, 1)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, classnums)
                    # 訓練時にはバックプロパゲーションを実施
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # 統計情報
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == classnums)
                
            # サンプル数で割って平均を求める
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('\t{} Loss: {:.4f} Acc: {:.4f} Time: {:.4f}'.format(phase, epoch_loss, epoch_acc, time.time()-end), end="")
            # Tensorboard 用に追加
            if phase == 'train':
                tbc.save_value('Loss', 'train_loss', epoch, epoch_loss)
                tbc.flush_line('train_loss')
            else:
                tbc.save_value('Loss', 'val_loss', epoch, epoch_loss)
                tbc.flush_line('val_loss')
                
            
            # 学習が進むに連れ、更新量を減らすための処理
            if phase == 'train':
                scheduler.step()
            # 精度が改善したらモデルを保存する
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            end = time.time()
            time.sleep(3) # 実行が速すぎるので、わざと休止を入れる
        print()

    time_elapsed = time.time() - since
    print()
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val acc: {:.4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

model = Net()
model = model.to(device)

学習の実施。学習中、先程のURLのページを眺めてください。30秒ごとに、学習経過が表示されます。
頻繁にチェックしすぎると、制限を超えるので、気をつけてください。

In [82]:
epochs = 25
batch_size = 32
lr = 0.001
momentum = 0.9
outdir = "."

# 損失関数（クロスエントロピー）、
# パラメータの最適化方法、学習率の更新方法を定義。
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.7)

# 実際の学習を実施
model = train_model(model, criterion, optimizer, exp_lr_scheduler, num_epochs=epochs)

tbc.close()

Net(
  (fc1): Linear(in_features=906, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=2, bias=True)
)

Epoch:0/24	train Loss: 0.6846 Acc: 0.5554 Time: 0.0424	val Loss: 0.6122 Acc: 0.6382 Time: 3.0099
Epoch:1/24	train Loss: 0.5308 Acc: 0.8154 Time: 3.0342	val Loss: 0.4746 Acc: 0.9347 Time: 3.0098
Epoch:2/24	train Loss: 0.4421 Acc: 0.8440 Time: 3.0375	val Loss: 0.3791 Acc: 0.9347 Time: 3.0102
Epoch:3/24	train Loss: 0.3311 Acc: 0.9161 Time: 3.0344	val Loss: 0.3211 Acc: 0.8995 Time: 3.0097
Epoch:4/24	train Loss: 0.2888 Acc: 0.8993 Time: 3.0360	val Loss: 0.4575 Acc: 0.7236 Time: 3.0098
Epoch:5/24	train Loss: 0.2828 Acc: 0.9044 Time: 3.0375	val Loss: 0.2334 Acc: 0.9548 Time: 3.0096
Epoch:6/24	train Loss: 0.2792 Acc: 0.8809 Time: 3.0377	val Loss: 0.2916 Acc: 0.9045 Time: 3.0099
Epoch:7/24	train Loss: 0.2022 Acc: 0.9379 Time: 3.0375	val Loss: 0.1832 Acc: 0.9598 Time: 3.0100
Epoch:8/24	train Loss: 0.2045 Acc: 0.9329 Time: 3.0374	val Loss: 0.2436 Acc: 0.8945 Time: 3.0099